# WEEK 3 COURSERA DATA SCIENCE CAPSTONE PROJECT

In [11]:
#Install beautifulsoup4 library
!conda install beautifulsoup4 bs4 --yes
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.



In [30]:
#instal library and web scraping
import urllib.request
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
article = page.read().decode()

with open('Wikipedia_canada.html', 'w') as fo:
    fo.write(article)

In [18]:
#Install lxml and html
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge/label/gcc7 lxml  --yes
!conda install -c conda-forge/label/cf201901 lxml --yes
!conda install -c conda-forge/label/cf202003 lxml --yes

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge/label/gcc7
    certifi-2018.8.24          |        py36_1001         139 KB  conda-forge/label/gcc7
    lxml-4.3.0                 |py36h23eabaa_1000         1.5 MB  conda-forge/label/gcc7
    ------------------------------------------------------------
                                           Total:         1.8 MB

The following packages will be UPDATED:

  lxml                       conda-forge::lxml-3.8.0-py36_0 --> conda-forge/label/gcc7::lxml-4.3.0-py36h23eabaa_1000

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-cert

In [55]:
#create dataFrame from wikipedia table
article = open('Wikipedia_canada.html').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')
df = (pd.read_html(str(soup.table))[0])  
df.shape

(180, 3)

In [56]:
#drop borough that have not assigned value
dropBorough = df[ df['Borough'] == 'Not assigned' ].index
df.drop(dropBorough , inplace=True)
df = df.reset_index(drop=True)
df.shape

(103, 3)

In [60]:
#Check if Neighborhood have "Not Assigned" value
df['Neighborhood'].unique()
'Not assigned' in df.Neighborhood.values


False

In [61]:
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [62]:
!wget -q -O 'Canada_geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [63]:
Geo_df = pd.read_csv('Canada_geospatial_data.csv')
Geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [65]:
df_canada = pd.merge(df, Geo_df, on='Postal Code')
df_canada.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [66]:
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [70]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [72]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, neighborhood in zip(df_canada['Latitude'], df_canada['Longitude'], df_canada['Borough'], df_canada['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### FINAL PROJECT for WEEK 3 FINISH